This is the cretin main file where using the functionality of the generator object class and the to generator class easy to read cretin code can be written 

In [40]:
from importlib import reload
import generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools #these python classes should be in the same folder as cretin_main
for obj in [generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools]:
    reload(obj)

import numpy as np
# These are all the functions inside the generator object class
gen_object = [i for i in dir(generator_object.User_input()) if i[0:2] != '__']
#print(gen_object)

# all the function inside the to_generator file 
gen_to_file = [i for i in dir(to_generator_string.Text_generator(generator_object.User_input())) if i[0:2] != '__']
#print(gen_to_file)

In [84]:
# This allows us to find frequntly used terms in the generator files, 
# To be specific the lines of text in the generator files get sliced on " " and then selected by the input, f.e 'alias'
# Then a histogram gets made of the most common results
# similarly for most used words in a chapter (ie "Radiation", "Geography" etc)

df = search.Generators().word(input = 'lasray', word_slicing = (0,10), print_path = False)
print(df.to_string())
#print(search.Generators().chapter(input = 'Controls'))

                             0
lasray 0.11 1. 0. 1.         4
lasray RLASER 1. 0. 1. FABS  1
               0
geometry      41
rlin          31
xlin          12
quad          10
rgeom          6
rlog           5
ylog           4
xlog           4
scale          4
product_mesh   4
xgeom          4
#endif         4
#ifdef         4
hex            3
scalekl        3
ylin           2
zlin           1
u3d            1
scaleklm       1
cone           1
velocity       1
wedge          1
u2d            1


In [42]:
# Here we reproduce lasfoam.gen from the test foder
gen2 = generator_object.User_input()

N0, N1 = 1, 21 #number of nodes is 20
E1 = 4.68
E2 = E1*1.001
rho = 0.003

gen2.materials_atom(index = 1, quantum_n_max = 3, element= "h")
gen2.materials_atom(index = 2, quantum_n_max = 3, element= "c")
gen2.materials_region(nodes = [N0, N1], elec_temp = 2.1)
gen2.materials_region_rho(rho)
gen2.materials_region_element(index = 1, initial_ion_population = 1)
gen2.materials_region_element(index = 2, initial_ion_population = 1)

gen2.geometry_nodes(coordinate= "x", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,10])

gen2.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(-5))
gen2.radiation_angles(n_rays = 3)

gen2.source_laser(laser_wavelength = 4, option_1= 'rate', option_2= 'history', laser_intensities = [1,1], nodes= [N0, N1])
gen2.source_jnu(E_range=[E1,E2], option_1= 'rate', option_2= 'history', nodes= [N0, N1])
gen2.source_jbndry(index = 1, E_range=[E1,E2], option_1= 'rate', option_2= 'history', nodes = [1,1], values=[1,2])

gen2.popular_switches(timestep_between_snapshot = 5)

gen2.controls(t_start=0., t_end=3*10**-6, restart = True)

write_run_plot.all(name = 'trial2', object = gen2, longprint=False, plot_duplicates=False)


writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial2/trial2.gen
running cretin with trial2
plotting trial2 too /home/brewster/Desktop/cretin.v2_19_test/test/trial2/images
plots trial2 finished at location /home/brewster/Desktop/cretin.v2_19_test/test/trial2/images


In [83]:
#John's asignment


# first we create our generic object
def sim(var):

    gen2 = generator_object.User_input()

    rho, T_ev, Tkelvin, ne , z = 6.9, 30, 512,  10**20, 12
    N0, N1 = 1, 41 #We are operating in 1d wih 20 nodes
    tin_n = 50
    E1, E2 = 4, 4.7 # the laser energies


    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "Sn")
    gen2.materials_region(nodes = [N0, N1], elec_temp = T_ev)
    gen2.materials_region_rho(rho)
    gen2.materials_region_background(ion_density = ne, electron_density = ne, average_charge = z, average_charge_squared=z**2, avg_atomic_number = tin_n )
    gen2.materials_region_element(index = 1, initial_ion_population = ne )
    gen2.geometry_nodes(coordinate= "x", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,10])
    gen2.radiation_ebins(n_boundaries = 61, start = 10**-5, end = 10**(5))
    gen2.radiation_angles(n_rays = 1)
    gen2.controls(t_start=0., t_end=3*10**-6, restart = True)
    gen2.popular_switches(timestep_between_snapshot = 5)
    #gen2.source_laser(laser_wavelength = 4, option_1= 'rate', option_2= 'history', laser_intensities = [1,1], nodes= [N0, N1])
    #gen2.source_jnu(E_range=[E1,E2], option_1= 'rate', option_2= 'history', nodes= [N0, var])
    #gen2.source_jbndry(index = 1, E_range=[E1,E2], option_1= 'rate', option_2= 'history', nodes = [1,1], values=[1,2])

    return gen2

i0, iend = 1,5
for i in range(i0,iend):

    variable = 1+2**i
    # adding our variable to the simulation
    gen2 = sim(variable)
    write_run_plot.all(name = 'trial'+str(i), object = gen2, longprint = False, plot_duplicates = False)

    if i>i0:
        org_trial, curr_trial = f'trial{str(i0)}',f'trial{str(i)}'
        # comparing data from the data dump for diffrent simulations with dependancy on 'variable'
        compare_dict = serial_sim_tools.check_dependancy(org_trial, curr_trial)
        val_lis = list(compare_dict.values())
        print(f'comparison of {org_trial} and {curr_trial}, number of identical arrays: {val_lis.count(True)} number of changed arrays: {val_lis.count(False)} number of comparision errors: {val_lis.count("comparison error")}')


writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial1/trial1.gen
running cretin with trial1
plotting trial1 too /home/brewster/Desktop/cretin.v2_19_test/test/trial1/images
plots trial1 finished at location /home/brewster/Desktop/cretin.v2_19_test/test/trial1/images

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial2/trial2.gen
running cretin with trial2
plotting trial2 too /home/brewster/Desktop/cretin.v2_19_test/test/trial2/images
plots trial2 finished at location /home/brewster/Desktop/cretin.v2_19_test/test/trial2/images
comparison of trial1 and trial2, number of identical arrays: 133 number of changed arrays: 0 number of comparision errors: 0

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial3/trial3.gen
running cretin with trial3
plotting trial3 too /home/brewster/Desktop/cretin.v2_19_test/test/trial3/images
plots trial3 finished at location /home/brewster/Desktop/cretin.v2_19_test/test/trial3/images
comparison of trial1 and trial3, number o

In [45]:
# reproducing detecterz.gen
gen1 = generator_object.User_input()

k1, k2 = 1, 21
l1, l2 = 1, 11
n2 = (k2*l2)/2 
NI, NE, TE = 10**20, 10**20, 10**3
R0, R1, Z0, Z1  = 0. , 1., -5, 5
RD, ZD = R1 + .1, Z1 +.1
MU1 = 10**-3
MU2 = -1 + MU1
PHI1, PHI2, PHI3 = 1.57, .785, .524

gen1.materials_region(nodes = [k1, k2, l1, l2], elec_temp = 1.1)
gen1.materials_region_background(k1, k2, l1, l2, TE)
gen1.geometry(type = 'rz')
gen1.geometry_quad(node_1 = [k1, k2], node_2= [l1, l2], x_cors = [R0, R1], y_cors = [Z0, Z1], ratios = [1,1])
gen1.radiation_ebins(n_boundaries = 50, start = 100, end = 1000)
gen1.radiation_angles(n_rays=3, n_angles= 15)
gen1.controls(t_start=0, t_end=0)
gen1.popular_switches(continuum_transfer='do steady-state continuum transfer',include_degeneracy='include electron degeneracy')

write_run_plot.all(name = 'trial1', object = gen1, longprint= False, plot_duplicates= False)




writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial1/trial1.gen
running cretin with trial1
plotting trial1 too /home/brewster/Desktop/cretin.v2_19_test/test/trial1/images
emis_0 has dimension 3 and has not been
jnu_0 has dimension 3 and has not been
kappa_0 has dimension 3 and has not been
plots trial1 finished at location /home/brewster/Desktop/cretin.v2_19_test/test/trial1/images
